# Automatic Record Merge in HubSpot

Nosotros realizamos un merge automatico de registros en HubSpot mediante la implementación de la herramienta de "Merge records" de HubSpot, por lo cual el merge se rige bajo las siguientes normas:

**Doc:** https://knowledge.hubspot.com/crm-setup/merge-records

Esta implementación de la herramienta de HubSpot la realizamo a traves de la API.

Este codigo trabaja haciendo uso del "Key" de los duplicados que obtenemos despues de implementar las funciones de encontrar duplicados.

# Set environment variables

In [28]:
import os

In [29]:
os.environ["path"] = r'C:\Users\Andres Garcia Bayona\Documents\Github\PyDev-Operational-Library-For-Data-Engineers'
os.environ["access_token"] = ''
os.environ["records_object_type"] = 'contacts'
os.environ["file_path"] = r'C:\Users\Andres Garcia Bayona\Documents\Training\APIs, CRMs and notebook management\Challenge3'
os.environ["file_name"] = 'Duplicate Contact Records Found by Phone Number.xlsx'

## Libraries

In [30]:
import requests
import json

import pandas as pd
import numpy as np

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

from IPython.display import display, HTML, clear_output

#### Data team library

Your library path

In [31]:
path = os.getenv('path')

In [32]:
import sys
sys.path.insert(0,path)

from functions.data_transformations.delete_unnecessary_blank_spaces import delete_unnecessary_blank_spaces

## Parameters - Input Values

#### Access API

In [33]:
access_token = os.getenv('access_token') # Input

headers = {'Content-Type': 'application/json',
            'authorization': 'Bearer {}'.format(access_token)}

#### Info Records

In [34]:
records_object_type = os.getenv('records_object_type')

#### Frame

In [35]:
file_path = os.getenv('file_path')

In [36]:
file_name = os.getenv('file_name')

In [37]:
df = pd.read_excel(file_path+'\\'+file_name)

In [38]:
df = df.replace(np.nan, '')

In [39]:
printmd("<h3><span style='color:blue'>You will work with {} records</span></h3>".format(len(df)))

<h3><span style='color:blue'>You will work with 113 records</span></h3>

In [41]:
df.head(5)

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key
0,2459,132924337805,Tyson,Newman,,+(353)6835 027381,"Balham Vale, 5116",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
1,317,132924473401,Tyson,Newman,tyson_newman1021718471+103@yahoo.com,+(353)6835 027381,"Apollo Crossroad, 6027",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
2,6607,132915056185,Tyson,Newman,,+(353)6835 027381,"Eldon Lane, 7122",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
3,561,132916656022,Sebastian,Parker,sebastian_parker1458479109+102@joiniaa.com,+(44)5517 227040,"Arbutus Crossroad, 4836",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
4,286,132911795531,Sebastian,Parker,sebastian_parker1458479109@joiniaa.com,+(44)5517 227040,"Bales Grove, 8324",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040


## Empty Keys

Vamos a realizar una pequeña revisión de las Keys, no debe existir ninguna key que este en blanco, realizar un merge de registros con keys vacias podria incluso eliminar bases de datos completas.

In [42]:
df['Key'] = df['Key'].apply(delete_unnecessary_blank_spaces)

In [43]:
detect_empty_keys = df.loc[df['Key'] == '']

if len(detect_empty_keys) != 0:
    printmd("<h3><span style='color:red'>There are keys configured as empty</span></h3> please check these keys that are empty [''], otherwise you will end up making merges that should not be made.")
else:
     printmd("<h3><span style='color:green'>Keys are properly configured</span></h3> Go ahead!")

<h3><span style='color:green'>Keys are properly configured</span></h3> Go ahead!

In [44]:
detect_empty_keys

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key


## Define number of duplicates

En esta longitud de valores seremos capaces de recorrer todo el frame de duplicados, obteniendo todas las key unicas que nos señalan los duplicados.

In [45]:
df = df.sort_values(['Key','Email'],ascending=[False, False])
df

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key
1,317,132924473401,Tyson,Newman,tyson_newman1021718471+103@yahoo.com,+(353)6835 027381,"Apollo Crossroad, 6027",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
0,2459,132924337805,Tyson,Newman,,+(353)6835 027381,"Balham Vale, 5116",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
2,6607,132915056185,Tyson,Newman,,+(353)6835 027381,"Eldon Lane, 7122",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
4,286,132911795531,Sebastian,Parker,sebastian_parker1458479109@joiniaa.com,+(44)5517 227040,"Bales Grove, 8324",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
5,559,132914154500,Sebastian,Parker,sebastian_parker1458479109+104@joiniaa.com,+(44)5517 227040,"Dutton Boulevard, 4639",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
...,...,...,...,...,...,...,...,...,...
107,4769,132910823864,Anthony,Sinclair,,+(353)3505 033457,"Besson Rue, 4781",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
108,6620,132911703138,Anthony,Sinclair,,+(353)3505 033457,"Baylis Grove, 980",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
109,6766,132915666010,Anthony,Sinclair,,+(353)3505 033457,"Chatsworth Grove, 8137",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
111,823,132911299715,Alan,Warner,alan_warner22954315@fuliss.net,+(44)7260 282170,"Bede Avenue, 4773",https://app.hubspot.com/contacts/50189227/reco...,alanwarner7260282170


In [46]:
duplicates = df.drop_duplicates(subset=['Key'])
duplicates = duplicates.reset_index(drop=True)

In [47]:
duplicates

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key
0,317,132924473401,Tyson,Newman,tyson_newman1021718471+103@yahoo.com,+(353)6835 027381,"Apollo Crossroad, 6027",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
1,286,132911795531,Sebastian,Parker,sebastian_parker1458479109@joiniaa.com,+(44)5517 227040,"Bales Grove, 8324",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
2,618,132919827222,Ryan,Welsch,ryan_welsch107397896+104@famism.biz,+(44)2778 635604,"Collingwood Road, 454",https://app.hubspot.com/contacts/50189227/reco...,ryanwelsch2778635604
3,334,132914194701,Ryan,Upsdell,ryan_upsdell175534425@vetan.org,+(44)3084 734307,"Bury Rue, 5932",https://app.hubspot.com/contacts/50189227/reco...,ryanupsdell3084734307
4,641,132920811942,Rocco,Palmer,rocco_palmer160419466@infotech44.tech,+(44)2813 405223,"Bagford Hill, 4868",https://app.hubspot.com/contacts/50189227/reco...,roccopalmer2813405223
5,42,132914893711,Rocco,Martin,rocco_martin507925367@eirey.tech,+(44)6084 412351,"Wager Street, 6624",https://app.hubspot.com/contacts/50189227/reco...,roccomartin6084412351
6,614,132914893703,Ramon,Preston,ramon_preston2096761104+103@sheye.org,+(353)4308 156553,"Dyott Vale, 9007",https://app.hubspot.com/contacts/50189227/reco...,ramonpreston4308156553
7,478,132913894432,Phillip,Cooper,phillip_cooper2140654386+104@jiman.org,+(44)6020 264200,"Argyle Lane, 9868",https://app.hubspot.com/contacts/50189227/reco...,phillipcooper6020264200
8,304,132924409170,Owen,Wigley,owen_wigley177952980@extex.org,+(353)8027 13401,"Elystan Drive, 8938",https://app.hubspot.com/contacts/50189227/reco...,owenwigley802713401
9,61,132914194705,Noah,Tanner,noah_tanner17953585+103@acrit.org,+(44)3156 686087,"Walnut Vale, 1387",https://app.hubspot.com/contacts/50189227/reco...,noahtanner3156686087


In [48]:
df

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key
1,317,132924473401,Tyson,Newman,tyson_newman1021718471+103@yahoo.com,+(353)6835 027381,"Apollo Crossroad, 6027",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
0,2459,132924337805,Tyson,Newman,,+(353)6835 027381,"Balham Vale, 5116",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
2,6607,132915056185,Tyson,Newman,,+(353)6835 027381,"Eldon Lane, 7122",https://app.hubspot.com/contacts/50189227/reco...,tysonnewman6835027381
4,286,132911795531,Sebastian,Parker,sebastian_parker1458479109@joiniaa.com,+(44)5517 227040,"Bales Grove, 8324",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
5,559,132914154500,Sebastian,Parker,sebastian_parker1458479109+104@joiniaa.com,+(44)5517 227040,"Dutton Boulevard, 4639",https://app.hubspot.com/contacts/50189227/reco...,sebastianparker5517227040
...,...,...,...,...,...,...,...,...,...
107,4769,132910823864,Anthony,Sinclair,,+(353)3505 033457,"Besson Rue, 4781",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
108,6620,132911703138,Anthony,Sinclair,,+(353)3505 033457,"Baylis Grove, 980",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
109,6766,132915666010,Anthony,Sinclair,,+(353)3505 033457,"Chatsworth Grove, 8137",https://app.hubspot.com/contacts/50189227/reco...,anthonysinclair3505033457
111,823,132911299715,Alan,Warner,alan_warner22954315@fuliss.net,+(44)7260 282170,"Bede Avenue, 4773",https://app.hubspot.com/contacts/50189227/reco...,alanwarner7260282170


## Merge Records

In [49]:
for i in range(0, len(duplicates)):
    
    clear_output(wait=True)
    
    print('Loop # {} of {}'.format(i, len(duplicates)-1))
    
    ## Our key to detect duplicates
    key = duplicates.at[i, 'Key']
    
    ## Find Duplicates Frame
    mini = df.loc[(df['Key'] == key)]
    
    ## To select the newest or oldest record as primary_record_id [ascending=True/False]
    #mini = mini.sort_values(by='Create Date', ascending=True) 
    mini = mini.reset_index(drop=True)
    
    # # # Loop for merging
    primary_record_id = int(mini.at[0, 'Record ID']) # Start record
    
    for j in range(1, len(mini)): #Start in 1 because primary_record_id will be the first [i = 0] in the start
        
        to_merge_record_id = int(mini.at[j, 'Record ID'])
        
        payload = json.dumps({'primaryObjectId': primary_record_id,
                              'objectIdToMerge': to_merge_record_id})
        
        url = 'https://api.hubapi.com/crm/v3/objects/{}/merge'.format(records_object_type)
        
        api_response = requests.request("POST", url, data=payload, headers=headers)
        
        ## El resultado del merge es un nuevo registro con un nuevo ID, este se convierte en el primary_record_id
        primary_record_id = api_response.json()['id']
        
        print('Merge # {} - Merging {} into {}'.format(j, to_merge_record_id, primary_record_id))
        print(api_response)
        
    # break ## Puse este break-loop_i para que revise el primer resultado que le dio - recuerde que desmerge no hay

Loop # 39 of 39
Merge # 1 - Merging 132910823976 into 132940301385
<Response [200]>


In [25]:
df

,Unnamed: 0,Record ID,First Name,Last Name,Email,Phone Number,Street Address,Record View,Key
1,650,132701233001,Tyson,Newman,tyson_newman1021718471+103@yahoo.com,+(353)6835 027381,"Apollo Crossroad, 6027",https://app.hubspot.com/contacts/43637626/reco...,tysonnewman6835027381
0,535,132689368629,Tyson,Newman,,+(353)6835 027381,"Balham Vale, 5116",https://app.hubspot.com/contacts/43637626/reco...,tysonnewman6835027381
2,6301,132687044360,Tyson,Newman,,+(353)6835 027381,"Eldon Lane, 7122",https://app.hubspot.com/contacts/43637626/reco...,tysonnewman6835027381
3,1223,132691310432,Sebastian,Parker,sebastian_parker1458479109@joiniaa.com,+(44)5517 227040,"Bales Grove, 8324",https://app.hubspot.com/contacts/43637626/reco...,sebastianparker5517227040
4,2963,132688588168,Sebastian,Parker,sebastian_parker1458479109+104@joiniaa.com,+(44)5517 227040,"Dutton Boulevard, 4639",https://app.hubspot.com/contacts/43637626/reco...,sebastianparker5517227040
...,...,...,...,...,...,...,...,...,...
124,6302,132678801316,Anthony,Sinclair,,+(353)3505 033457,"Baylis Grove, 980",https://app.hubspot.com/contacts/43637626/reco...,anthonysinclair3505033457
125,6266,132685725964,Anthony,Sinclair,,+(353)3505 033457,"Besson Rue, 4781",https://app.hubspot.com/contacts/43637626/reco...,anthonysinclair3505033457
126,6076,132679269222,Anthony,Sinclair,,+(353)3505 033457,"Chatsworth Grove, 8137",https://app.hubspot.com/contacts/43637626/reco...,anthonysinclair3505033457
129,1431,132700294836,Alan,Warner,alan_warner22954315@fuliss.net,+(44)7260 282170,"Bede Avenue, 4773",https://app.hubspot.com/contacts/43637626/reco...,alanwarner7260282170
